In [7]:
import pathlib

import numpy as np
import pandas as pd
import tqdm

In [8]:
CHUNKSIZE = 1e4

In [9]:
base_df = pd.read_csv(
    "../data/База по всем организациям и ИП.zip",
    sep=";", 
    chunksize=CHUNKSIZE, 
    dtype=str
)

In [10]:
out_file = pathlib.Path("/home/pavel/search_base.csv")

for chunk in tqdm.tqdm(base_df):
    try:
        chunk.dropna(subset=["Наименование полное", "ИНН"], how="any", inplace=True)
        chunk = chunk.loc[chunk["Головная компания (1) или филиал (0)"] == '1', :].copy()
        
        chunk["name"] = chunk["Наименование полное"].str.upper()
        chunk["tax_number"] = chunk["ИНН"]
        chunk["individual"] = chunk["ОКОПФ (расшифровка)"] == "Индивидуальные предприниматели"
        chunk["creation_date"] = pd.to_datetime(chunk["Дата создания"], errors="coerce").dt.date
        chunk["active"] = chunk["Компания действующая (1) или нет (0)"].fillna(0).astype(bool)
        chunk["activity_code"] = chunk["ОКВЭД2"]
        chunk["legal_address"] = chunk["Юр адрес"]
        chunk["fact_address"] = chunk["Факт адрес"]
        
        chunk = chunk[[
            "name", "tax_number", "individual", "creation_date",
            "active", "activity_code", "legal_address", "fact_address"
        ]]

        if out_file.exists():
            chunk.to_csv(out_file, header=False, index=False, mode="a")
        else:
            chunk.to_csv(out_file, header=True, index=False, mode="w")
    except Exception as e:
        print(e)
        pass

3184it [16:28,  3.22it/s]
